In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LeakyReLU, ELU
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from scipy.stats import gamma
import os

np.random.seed(60)

# Function to generate synthetic data for Gamma distribution parameters (shape and scale)
def generate_gamma_data(num_shapes=200, num_scales=200):
    a_values = np.random.uniform(1, 5, num_shapes)
    b_values = np.random.uniform(0.5, 2, num_scales)

    x = []
    y_shape = []
    y_scale = []
    sample_sizes = np.random.randint(10, 100, 25)

    for a in a_values:
        for b in b_values:
            for num_samples in sample_sizes:
                for _ in range(5):
                    samples = np.random.gamma(shape=a, scale=b, size=num_samples)
                    samples = np.sort(samples)
                    x.append(np.insert(samples, 0, num_samples))
                    y_shape.append([a])
                    y_scale.append([b])

    max_length = 101
    x = pad_sequences(x, maxlen=max_length, dtype='float32', padding='post', truncating='post')

    x = np.array(x)
    y_shape = np.array(y_shape)
    y_scale = np.array(y_scale)

    return x, y_shape, y_scale

# Generate synthetic data for Gamma distribution
x, y_shape, y_scale = generate_gamma_data()

# Split the data into training and testing sets
x_train, x_test, y_train_shape, y_test_shape, y_train_scale, y_test_scale = train_test_split(
    x, y_shape, y_scale, test_size=0.2, random_state=42
)

# Save data to Google Drive (adjust the path as needed)
data_dir = "/content/drive/MyDrive/deep_learning"
data_path = os.path.join(data_dir, "data_large.npz")

os.makedirs(data_dir, exist_ok=True)

np.savez(data_path,
         x_train=x_train,
         x_test=x_test,
         y_train_shape=y_train_shape,
         y_train_scale=y_train_scale,
         y_test_shape=y_test_shape,
         y_test_scale=y_test_scale)
print(f"Training and test data saved to {data_path}")

# Define the neural network architecture
input_layer = Input(shape=(101,))
hidden1 = Dense(101, kernel_regularizer=l1(0.01))(input_layer)
hidden1 = ELU(alpha=1.0)(hidden1)
hidden2 = Dense(101, kernel_regularizer=l2(0.01))(hidden1)
hidden2 = LeakyReLU(alpha=0.01)(hidden2)
hidden3 = Dense(101, kernel_regularizer=l1(0.01))(hidden2)
hidden3 = ELU(alpha=1.0)(hidden3)
hidden4 = Dense(101, kernel_regularizer=l2(0.01))(hidden3)
hidden4 = LeakyReLU(alpha=0.01)(hidden4)

# Branch for shape (a)
shape_hidden5 = Dense(101, kernel_regularizer=l1(0.01))(hidden4)
shape_hidden5 = ELU(alpha=1.0)(shape_hidden5)
shape_hidden6 = Dense(101, kernel_regularizer=l2(0.01))(shape_hidden5)
shape_hidden6 = LeakyReLU(alpha=0.01)(shape_hidden6)
shape_hidden7 = Dense(10, kernel_regularizer=l1(0.01))(shape_hidden6)
shape_hidden7 = ELU(alpha=1.0)(shape_hidden7)
shape_hidden8 = Dense(10, kernel_regularizer=l2(0.01))(shape_hidden7)
shape_hidden8 = LeakyReLU(alpha=0.01)(shape_hidden8)
shape_hidden9 = Dense(10, kernel_regularizer=l1(0.01))(shape_hidden8)
shape_hidden9 = ELU(alpha=1.0)(shape_hidden9)
shape_hidden10 = Dense(10, kernel_regularizer=l2(0.01))(shape_hidden9)
shape_hidden10 = LeakyReLU(alpha=0.01)(shape_hidden10)
shape_hidden11 = Dense(10, kernel_regularizer=l1(0.01))(shape_hidden10)
shape_hidden11 = ELU(alpha=1.0)(shape_hidden11)
shape_hidden12 = Dense(10, kernel_regularizer=l2(0.01))(shape_hidden11)
shape_hidden12 = LeakyReLU(alpha=0.01)(shape_hidden12)
shape_hidden13 = Dense(10, kernel_regularizer=l1(0.01))(shape_hidden12)
shape_hidden13 = ELU(alpha=1.0)(shape_hidden13)
shape_hidden14 = Dense(10, kernel_regularizer=l2(0.01))(shape_hidden13)
shape_hidden14 = LeakyReLU(alpha=0.01)(shape_hidden14)
shape_output = Dense(1, activation='linear', kernel_regularizer=l2(0.01), name='shape_output')(shape_hidden14)

# Branch for scale (b)
scale_hidden5 = Dense(101, kernel_regularizer=l1(0.01))(hidden4)
scale_hidden5 = ELU(alpha=1.0)(scale_hidden5)
scale_hidden6 = Dense(101, kernel_regularizer=l2(0.01))(scale_hidden5)
scale_hidden6 = LeakyReLU(alpha=0.01)(scale_hidden6)
scale_hidden7 = Dense(10, kernel_regularizer=l1(0.01))(scale_hidden6)
scale_hidden7 = ELU(alpha=1.0)(scale_hidden7)
scale_hidden8 = Dense(10, kernel_regularizer=l2(0.01))(scale_hidden7)
scale_hidden8 = LeakyReLU(alpha=0.01)(scale_hidden8)
scale_output = Dense(1, activation='linear', kernel_regularizer=l2(0.01), name='scale_output')(scale_hidden8)

# Define the model
model = Model(inputs=input_layer, outputs=[shape_output, scale_output])

# Compile the model with separate losses for each output
model.compile(
    optimizer='adam',
    loss={'shape_output': 'mean_squared_error', 'scale_output': 'mean_squared_error'},
    metrics={'shape_output': ['mean_squared_error'], 'scale_output': ['mean_squared_error']}
)

# Define callbacks
checkpoint_path = os.path.join(data_dir, "model_checkpoint.h5")
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    save_best_only=True,
    save_freq='epoch'
)

early_stopping_callback = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)
if os.path.exists(checkpoint_path):
    model.load_weights(checkpoint_path)
    print(f"Loaded weights from {checkpoint_path}")
else:
    print(f"No checkpoint found at {checkpoint_path}. Training from scratch.")

# Train the model
history = model.fit(
    x_train, {'shape_output': y_train_shape, 'scale_output': y_train_scale},
    validation_data=(x_test, {'shape_output': y_test_shape, 'scale_output': y_test_scale}),
    epochs=1000, batch_size=32,
    callbacks=[checkpoint_callback, early_stopping_callback]
)

# Save the final model weights
model.save_weights(checkpoint_path)

# Evaluate the model
evaluation = model.evaluate(x_test, {'shape_output': y_test_shape, 'scale_output': y_test_scale})
loss, mse_shape, mse_scale = evaluation[0], evaluation[1], evaluation[2]
print(f"Test Loss: {loss}, Test MSE for Shape: {mse_shape}, Test MSE for Scale: {mse_scale}")

# Predictions
predictions = model.predict(x_test)
predicted_shapes = predictions[0]
predicted_scales = predictions[1]

print("Sample predictions:")
for i in range(5):
    print(f"Predicted Shape (a): {predicted_shapes[i][0]}, Actual Shape (a): {y_test_shape[i][0]}")
    print(f"Predicted Scale (b): {predicted_scales[i][0]}, Actual Scale (b): {y_test_scale[i][0]}")

# Compare with MLE
def gamma_mle(samples):
    shape, loc, scale = gamma.fit(samples, floc=0)
    return shape, scale

print("\nComparison with MLE:")
for i in range(5):
    sample = x_test[i][1:1 + int(x_test[i][0])]
    mle_shape, mle_scale = gamma_mle(sample)
    print(f"Sample {i+1}: MLE Shape: {mle_shape:.4f}, Predicted Shape: {predicted_shapes[i][0]:.4f}")
    print(f"Sample {i+1}: MLE Scale: {mle_scale:.4f}, Predicted Scale: {predicted_scales[i][0]:.4f}")


Mounted at /content/drive
Training and test data saved to /content/drive/MyDrive/deep_learning/data_large.npz
Loaded weights from /content/drive/MyDrive/deep_learning/model_checkpoint.h5
Epoch 1/1000
125000/125000 [==============================] - 804s 6ms/step - loss: 0.7466 - shape_output_loss: 0.3208 - scale_output_loss: 0.1622 - shape_output_mean_squared_error: 0.3208 - scale_output_mean_squared_error: 0.1622 - val_loss: 0.7274 - val_shape_output_loss: 0.3127 - val_scale_output_loss: 0.1582 - val_shape_output_mean_squared_error: 0.3127 - val_scale_output_mean_squared_error: 0.1582
Epoch 2/1000
125000/125000 [==============================] - 749s 6ms/step - loss: 0.7385 - shape_output_loss: 0.3179 - scale_output_loss: 0.1595 - shape_output_mean_squared_error: 0.3179 - scale_output_mean_squared_error: 0.1595 - val_loss: 0.7313 - val_shape_output_loss: 0.3104 - val_scale_output_loss: 0.1581 - val_shape_output_mean_squared_error: 0.3104 - val_scale_output_mean_squared_error: 0.1581
E